<a href="https://colab.research.google.com/github/OrtegaJulio7/SIS420/blob/main/Primer%20Parcial/1parcial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Primer parcial ( Regresion one vs all)
### Nombre: Ortega Albarado Julio Cesar
### Carrera: Ing. de Sistemas
### Dataset: https://www.kaggle.com/datasets/colewelkins/cardiovascular-disease
### Github:

In [57]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np

# Libreria para graficos
from matplotlib import pyplot

# Modulo de optimizacion en scipy
from scipy import optimize

# modulo para cargar archivos en formato MATLAB
# from scipy.io import loadmat

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

Variables

ID : Identificador único para cada paciente.

edad : Edad del paciente en días.

age_years : Edad del paciente en años (derivada de age).

género : género del paciente. Variable categórica (1: femenino, 2: masculino).

altura : Altura del paciente en centímetros.

peso : Peso del paciente en kilogramos.

ap_hi : Presión arterial sistólica.

ap_lo : Presión arterial diastólica.

Colesterol : Niveles de colesterol. Variable categórica (1: Normal, 2: Por encima de lo normal, 3: Muy por encima de lo normal).

gluc : Niveles de glucosa. Variable categórica (1: Normal, 2: Por encima de lo normal, 3: Muy por encima de lo normal).

smoke : Estado de fumador. Variable binaria (0: no fumador, 1: fumador).

alco : Consumo de alcohol. Variable binaria (0: No consume alcohol, 1: Consume alcohol).

activo : Actividad física. Variable binaria (0: No físicamente activo, 1: Físicamente activo).

cardio : Presencia o ausencia de enfermedad cardiovascular.Variable objetivo. Binaria (0: Ausencia, 1: Presencia).

imc : Índice de masa corporal, derivado de weighty height. Calculado como ( \text{IMC} = \frac{\text{peso (kg)}}{\text{altura (m)}^2} ).

bp_category : Categoría de presión arterial basada en ap_hiy ap_lo. Las categorías incluyen "Normal", "Elevada", "Hipertensión en etapa 1", "Hipertensión en etapa 2" y "Crisis hipertensiva".

bp_category_encoded : Forma codificada bp_categorypara fines de aprendizaje automático.

atributos sintéticos de texto, caracteres o booleanos:

TEXTO

a. "Cardiovascular Risk Level":
Un resumen categórico que refleja el riesgo de enfermedad cardiovascular según factores como colesterol, glucosa, presión arterial, tabaquismo, alcohol, y actividad física.

Ejemplo de valores: ["Low", "Moderate", "High", "Very High"]
Criterios:
"Low" si colesterol y glucosa son normales, y no fuma ni bebe.
"Very High" si tiene hipertensión y es fumador.

BOOLEANOS

b. "Hypertension Risk":
Este atributo indica si el paciente tiene riesgo de hipertensión basado en su presión arterial y consumo de tabaco o alcohol.

CARACTER

a. "Lifestyle":
Un atributo que combina varios factores para describir el estilo de vida del paciente (actividad física, consumo de alcohol, tabaquismo).

Ejemplo de valores: ["Healthy", "Moderate", "Unhealthy"]


In [59]:
import pandas as pd
import numpy as np

# 4 etiquetas, de 0 a 3 (normal = 0, hypertension1 = 1, hypertension2 = 2, Elevated: 3)
num_labels = 4


# Cargar los datos usando Pandas
df = pd.read_csv('/content/drive/MyDrive/datasets/cardio_data_processed.csv')

#--------------
# Condiciones para el riesgo cardiovascular
conditions = [
    (df['cholesterol'] == 1) & (df['bp_category'] == 'Normal') & (df['smoke'] == 0) & (df['alco'] == 0),
    (df['cholesterol'] == 2) | (df['gluc'] == 2) | (df['bp_category'] == 'Elevated'),
    (df['cholesterol'] == 3) | (df['gluc'] == 3) | (df['bp_category'] == 'Hypertension Stage 1'),
    (df['cardio'] == 1) | (df['bp_category'] == 'Hypertensive Crisis')
]
risk_labels = ['Low', 'Moderate', 'High', 'Very High']
df['Cardiovascular_Risk'] = np.select(conditions, risk_labels, default='Moderate')

#-Este atributo indica si el paciente tiene riesgo de hipertensión basado en su presión arterial y consumo de tabaco o alcohol.
df['Hypertension_Risk'] = (df['ap_hi'] > 130) | (df['smoke'] == 1) | (df['alco'] == 1)

#Un atributo que combina varios factores para describir el estilo de vida del paciente (actividad física, consumo de alcohol, tabaquismo)
conditions = [
    (df['active'] == 1) & (df['smoke'] == 0) & (df['alco'] == 0),
    (df['active'] == 1) & ((df['smoke'] == 1) | (df['alco'] == 1)),
    (df['active'] == 0) & ((df['smoke'] == 1) | (df['alco'] == 1))
]
lifestyle_labels = ['Healthy', 'Moderate', 'Unhealthy']
df['Lifestyle'] = np.select(conditions, lifestyle_labels, default='Moderate')
###########
#mapeo
risk_mapping = {
    'Low': 1,
    'Moderate': 2,
    'High': 3,
    'Very High': 4
}
df['Cardiovascular_Risk'] = df['Cardiovascular_Risk'].map(risk_mapping)

df['Hypertension_Risk'] = df['Hypertension_Risk'].astype(int)

lifestyle_mapping = {
    'Healthy': 1,
    'Moderate': 2,
    'Unhealthy': 3
}
df['Lifestyle'] = df['Lifestyle'].map(lifestyle_mapping)

#------------------
# df.to_csv('tu_dataset_actualizado.csv', index=False)
# Omitir la columna 0(no tiene relevancia por que es la numeración de las filas) en X y la columna 17(es una columna repetida de Y)
X_test = df.iloc[0:54500, 1:15].values
y_test = df.iloc[0:54500:, 16].values

X_train = df.iloc[54500:, 1:15].values
y_train = df.iloc[54500:, 16].values


X=X_test
y=y_test

#X=X_train
#y=y_train

# Mapear los valores de la columna 16 según la lógica (normal = 0, hypertension1 = 1, hypertension2 = 2, Elevated: 3)
mapping = {
    'Hypertension Stage 1': 1,
    'Hypertension Stage 2': 2,
    'Normal': 0,
    'Elevated': 3
}

# Reemplazar los valores en la columna de etiquetas según el diccionario de mapeo
y_mapped = np.array([mapping[val] for val in y])
y=y_mapped
y[y == 4] = 0



# print(X)

m = y.size

In [60]:
print(df.head())

   id    age  gender  height  weight  ap_hi  ap_lo  cholesterol  gluc  smoke  \
0   0  18393       2     168    62.0    110     80            1     1      0   
1   1  20228       1     156    85.0    140     90            3     1      0   
2   2  18857       1     165    64.0    130     70            3     1      0   
3   3  17623       2     169    82.0    150    100            1     1      0   
4   4  17474       1     156    56.0    100     60            1     1      0   

   alco  active  cardio  age_years        bmi           bp_category  \
0     0       1       0         50  21.967120  Hypertension Stage 1   
1     0       1       1         55  34.927679  Hypertension Stage 2   
2     0       0       1         51  23.507805  Hypertension Stage 1   
3     0       1       1         48  28.710479  Hypertension Stage 2   
4     0       0       0         47  23.011177                Normal   

    bp_category_encoded  Cardiovascular_Risk  Hypertension_Risk  Lifestyle  
0  Hypertension

In [61]:
df.dropna(axis=0, thresh=2)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,age_years,bmi,bp_category,bp_category_encoded,Cardiovascular_Risk,Hypertension_Risk,Lifestyle
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0,50,21.967120,Hypertension Stage 1,Hypertension Stage 1,3,0,1
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1,55,34.927679,Hypertension Stage 2,Hypertension Stage 2,3,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1,51,23.507805,Hypertension Stage 1,Hypertension Stage 1,3,0,2
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1,48,28.710479,Hypertension Stage 2,Hypertension Stage 2,4,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0,47,23.011177,Normal,Normal,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68200,99993,19240,2,168,76.0,120,80,1,1,1,0,1,0,52,26.927438,Hypertension Stage 1,Hypertension Stage 1,3,1,2
68201,99995,22601,1,158,126.0,140,90,2,2,0,0,1,1,61,50.472681,Hypertension Stage 2,Hypertension Stage 2,2,1,1
68202,99996,19066,2,183,105.0,180,90,3,1,0,1,0,1,52,31.353579,Hypertension Stage 2,Hypertension Stage 2,3,1,3
68203,99998,22431,1,163,72.0,135,80,1,2,0,0,0,1,61,27.099251,Hypertension Stage 1,Hypertension Stage 1,2,1,2


In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68205 entries, 0 to 68204
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   68205 non-null  int64  
 1   age                  68205 non-null  int64  
 2   gender               68205 non-null  int64  
 3   height               68205 non-null  int64  
 4   weight               68205 non-null  float64
 5   ap_hi                68205 non-null  int64  
 6   ap_lo                68205 non-null  int64  
 7   cholesterol          68205 non-null  int64  
 8   gluc                 68205 non-null  int64  
 9   smoke                68205 non-null  int64  
 10  alco                 68205 non-null  int64  
 11  active               68205 non-null  int64  
 12  cardio               68205 non-null  int64  
 13  age_years            68205 non-null  int64  
 14  bmi                  68205 non-null  float64
 15  bp_category          68205 non-null 

In [63]:
import numpy as np

# Crear una copia del dataset
df_synthetic = df.copy()

# Añadir pequeñas perturbaciones a variables numéricas dentro de un rango controlado
df_synthetic['age'] = df['age'] + np.random.randint(-365, 365, size=len(df))  # +/- 1 año
df_synthetic['height'] = df['height'] + np.random.uniform(-5, 5, size=len(df))  # +/- 5 cm
df_synthetic['weight'] = df['weight'] + np.random.uniform(-2, 2, size=len(df))  # +/- 2 kg

# Modificar presión arterial sistólica y diastólica con pequeñas variaciones
df_synthetic['ap_hi'] = df['ap_hi'] + np.random.randint(-5, 5, size=len(df))  # +/- 5 mmHg
df_synthetic['ap_lo'] = df['ap_lo'] + np.random.randint(-5, 5, size=len(df))  # +/- 5 mmHg

# Recalcular el IMC (BMI) basado en los nuevos valores de altura y peso
df_synthetic['bmi'] = df_synthetic['weight'] / (df_synthetic['height'] / 100) ** 2


In [64]:
# Recalcular la categoría de presión arterial basada en los valores modificados
def bp_category(ap_hi, ap_lo):
    if ap_hi < 120 and ap_lo < 80:
        return 'Normal'
    elif 120 <= ap_hi < 130 and ap_lo < 80:
        return 'Elevated'
    elif 130 <= ap_hi < 140 or 80 <= ap_lo < 90:
        return 'Hypertension Stage 1'
    elif ap_hi >= 140 or ap_lo >= 90:
        return 'Hypertension Stage 2'
    else:
        return 'Hypertensive Crisis'

df_synthetic['bp_category'] = df_synthetic.apply(lambda row: bp_category(row['ap_hi'], row['ap_lo']), axis=1)


In [65]:
# Cambiar estado de fumador, alcohol y actividad física con un 10% de probabilidad
df_synthetic['smoke'] = df['smoke'].apply(lambda x: x if np.random.rand() > 0.1 else 1 - x)
df_synthetic['alco'] = df['alco'].apply(lambda x: x if np.random.rand() > 0.1 else 1 - x)
df_synthetic['active'] = df['active'].apply(lambda x: x if np.random.rand() > 0.1 else 1 - x)


In [66]:
# Recalcular el IMC después de modificaciones en altura y peso
df_synthetic['bmi'] = df_synthetic['weight'] / (df_synthetic['height'] / 100) ** 2

# Recalcular la categoría codificada de presión arterial
bp_category_mapping = {
    'Normal': 0,
    'Elevated': 1,
    'Hypertension Stage 1': 2,
    'Hypertension Stage 2': 3,
    'Hypertensive Crisis': 4
}
df_synthetic['bp_category_encoded'] = df_synthetic['bp_category'].map(bp_category_mapping)


In [72]:
# Concatenar el dataset original y el dataset sintético
# 4 etiquetas, de 0 a 3 (normal = 0, hypertension1 = 1, hypertension2 = 2, Elevated: 3)
num_labels = 4

df_combined = pd.concat([df, df_synthetic], ignore_index=True)

#------------------
# Mover la columna 'bp_category' al final
cols = list(df_combined.columns)
cols.append(cols.pop(cols.index('bp_category')))
df_combined = df_combined[cols]

# Eliminar la columna 'bp_category_encoded'
df_combined = df_combined.drop(columns=['bp_category_encoded'])


# # Omitir la columna 0(no tiene relevancia por que es la numeración de las filas) en X y la columna 17(es una columna repetida de Y)
X_test = df_combined.iloc[0:95487, 1:17].values # prueba
y_test = df_combined.iloc[0:95487:, 18].values

X_train = df_combined.iloc[95487:, 1:17].values #entrenamiento
y_train = df_combined.iloc[95487:, 18].values


X=X_test
y=y_test

#X=X_train
#y=y_train

# Mapear los valores de la columna 16 según la lógica (normal = 0, hypertension1 = 1, hypertension2 = 2, Elevated: 3)
mapping = {
    'Hypertension Stage 1': 1,
    'Hypertension Stage 2': 2,
    'Normal': 0,
    'Elevated': 3
}

# Reemplazar los valores en la columna de etiquetas según el diccionario de mapeo
y_mapped = np.array([mapping[val] for val in y])
y=y_mapped
y[y == 4] = 0



# print(X)

m = y.size


In [73]:
print(X)
print(y)

[[1.83930000e+04 2.00000000e+00 1.68000000e+02 ... 2.19671202e+01
  3.00000000e+00 0.00000000e+00]
 [2.02280000e+04 1.00000000e+00 1.56000000e+02 ... 3.49276792e+01
  3.00000000e+00 1.00000000e+00]
 [1.88570000e+04 1.00000000e+00 1.65000000e+02 ... 2.35078053e+01
  3.00000000e+00 0.00000000e+00]
 ...
 [1.54100000e+04 1.00000000e+00 1.63743884e+02 ... 2.06190295e+01
  1.00000000e+00 0.00000000e+00]
 [1.97990000e+04 1.00000000e+00 1.69538389e+02 ... 2.33705378e+01
  3.00000000e+00 0.00000000e+00]
 [1.51430000e+04 2.00000000e+00 1.60460174e+02 ... 3.27102572e+01
  4.00000000e+00 1.00000000e+00]]
[1 2 1 ... 0 1 2]


In [76]:
df_combined.info()
df_combined.dropna(axis=0, thresh=2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136410 entries, 0 to 136409
Data columns (total 19 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   136410 non-null  int64  
 1   age                  136410 non-null  int64  
 2   gender               136410 non-null  int64  
 3   height               136410 non-null  float64
 4   weight               136410 non-null  float64
 5   ap_hi                136410 non-null  int64  
 6   ap_lo                136410 non-null  int64  
 7   cholesterol          136410 non-null  int64  
 8   gluc                 136410 non-null  int64  
 9   smoke                136410 non-null  int64  
 10  alco                 136410 non-null  int64  
 11  active               136410 non-null  int64  
 12  cardio               136410 non-null  int64  
 13  age_years            136410 non-null  int64  
 14  bmi                  136410 non-null  float64
 15  Cardiovascular_Ri

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,age_years,bmi,Cardiovascular_Risk,Hypertension_Risk,Lifestyle,bp_category
0,0,18393,2,168.000000,62.000000,110,80,1,1,0,0,1,0,50,21.967120,3,0,1,Hypertension Stage 1
1,1,20228,1,156.000000,85.000000,140,90,3,1,0,0,1,1,55,34.927679,3,1,1,Hypertension Stage 2
2,2,18857,1,165.000000,64.000000,130,70,3,1,0,0,0,1,51,23.507805,3,0,2,Hypertension Stage 1
3,3,17623,2,169.000000,82.000000,150,100,1,1,0,0,1,1,48,28.710479,4,1,1,Hypertension Stage 2
4,4,17474,1,156.000000,56.000000,100,60,1,1,0,0,0,0,47,23.011177,1,0,2,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136405,99993,19237,2,169.624032,74.441004,117,77,1,1,1,0,1,0,52,25.872444,3,1,2,Normal
136406,99995,22926,1,155.156248,124.906573,143,91,2,2,0,0,1,1,61,51.885590,2,1,1,Hypertension Stage 2
136407,99996,19265,2,186.608069,104.439866,179,85,3,1,0,1,1,1,52,29.992003,3,1,3,Hypertension Stage 1
136408,99998,22470,1,161.739747,72.061523,135,79,1,2,0,0,0,1,61,27.546721,2,1,2,Hypertension Stage 1


In [77]:
def  featureNormalize(X):
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

In [78]:
# llama featureNormalize con los datos cargados
X_norm, mu, sigma = featureNormalize(X)

In [79]:
# Configurar la matriz adecuadamente, y agregar una columna de unos que corresponde al termino de intercepción.
m, n = X.shape
# Agraga el termino de intercepción a A
# X = np.concatenate([np.ones((m, 1)), X_norm], axis=1)
X = X_norm
# X = np.concatenate([np.ones((m, 1)), X], axis=1)

In [80]:
# Selecciona aleatoriamente 100 puntos de datos para mostrar
rand_indices = np.random.choice(m, 100, replace=False)
sel = X[rand_indices, :]

# displayData(sel)

In [81]:
def sigmoid(z):
    """
    Calcula la sigmoide de z.
    """
    return 1.0 / (1.0 + np.exp(-z))

In [82]:
def lrCostFunction(theta, X, y, lambda_):
    """
    Calcula el costo de usar theta como parámetro para la regresión logística regularizada y
    el gradiente del costo w.r.t. a los parámetros.

    Parametros
    ----------
    theta : array_like
        Parametro theta de la regresion logistica. Vector de la forma(shape) (n, ). n es el numero de caracteristicas
        incluida la intercepcion

    X : array_like
        Dataset con la forma(shape) (m x n). m es el numero de ejemplos, y n es el numero de
        caracteristicas (incluida la intercepcion).

    y : array_like
        El conjunto de etiquetas. Un vector con la forma (shape) (m, ). m es el numero de ejemplos

    lambda_ : float
        Parametro de regularización.

    Devuelve
    -------
    J : float
        El valor calculado para la funcion de costo regularizada.

    grad : array_like
        Un vector de la forma (shape) (n, ) que es el gradiente de la
        función de costo con respecto a theta, en los valores actuales de theta..
    """
#     alpha = 0.003
#     theta = theta.copy()
    # Inicializa algunos valores utiles
    m = y.size

    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)

    J = 0
    grad = np.zeros(theta.shape)

    h = sigmoid(X.dot(theta.T))

    temp = theta
    temp[0] = 0

#     J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h)))
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))# factor de regularizacion (lambda_ / (2 * m)) * np.sum(np.square(temp))

    grad = (1 / m) * (h - y).dot(X)
#     theta = theta - (alpha / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp

    return J, grad
#    return J, theta

In [83]:
def oneVsAll(X, y, num_labels, lambda_):
    """
    Trains num_labels logistic regression classifiers and returns
    each of these classifiers in a matrix all_theta, where the i-th
    row of all_theta corresponds to the classifier for label i.

    Parameters
    ----------
    X : array_like
        The input dataset of shape (m x n). m is the number of
        data points, and n is the number of features. Note that we
        do not assume that the intercept term (or bias) is in X, however
        we provide the code below to add the bias term to X.

    y : array_like
        The data labels. A vector of shape (m, ).

    num_labels : int
        Number of possible labels.

    lambda_ : float
        The logistic regularization parameter.

    Returns
    -------
    all_theta : array_like
        The trained parameters for logistic regression for each class.
        This is a matrix of shape (K x n+1) where K is number of classes
        (ie. `numlabels`) and n is number of features without the bias.
    """
    # algunas variables utiles
    m, n = X.shape

    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 50}
        res = optimize.minimize(lrCostFunction,
                                initial_theta,
                                (X, (y == c), lambda_),
                                jac=True,
                                method='CG',
                                options=options)

        all_theta[c] = res.x

    return all_theta

In [84]:
lambda_ = 0.1
all_theta = oneVsAll(X, y, num_labels, lambda_)
print(all_theta.shape)

(4, 17)


In [85]:
print(all_theta)

[[-5.88722593e+00  2.91886052e-01 -2.53873748e-02  8.14874918e-02
  -8.85749288e-02 -4.41757299e+00 -2.20464900e+00  1.22257220e-02
  -2.43278108e-02  3.72832092e-01  2.73324674e-01 -9.24961062e-02
   6.42610177e-02 -2.73664168e-01  6.23764776e-02 -5.26347548e-01
  -6.10957767e-01]
 [ 1.86682727e-01  4.89382154e-02  5.75564907e-02  4.27247196e-02
   4.18427111e-02 -9.84458984e-02  1.58088697e-01  1.53723336e-01
   5.15528453e-02  2.44290766e-01  1.04049708e-01  1.54761560e-02
  -1.58288455e-01  4.60597409e-02  2.76918437e-02  9.10248439e-01
  -9.64685629e-01]
 [-3.16484258e+00 -8.41930394e-02 -1.46121646e-02  3.49191651e-02
  -4.18275243e-02  6.28445994e-01  2.38682408e+00 -4.07820730e-02
   1.79149449e-02 -2.46403591e-01 -1.62627552e-01  2.37221414e-02
  -3.94146705e-03  7.30384960e-02 -3.90834671e-03  2.89665747e-02
   1.04326234e+00]
 [-3.72657250e+00  1.53786826e-02  9.44664676e-02  1.36366413e-01
  -1.14780070e-01  1.56270592e+00 -2.02642412e+00 -1.20002952e-01
   3.58180291e-02  

In [86]:
def predictOneVsAll(all_theta, X):
    """
    Devuelve un vector de predicciones para cada ejemplo en la matriz X.
    Tenga en cuenta que X contiene los ejemplos en filas.
    all_theta es una matriz donde la i-ésima fila es un vector theta de regresión logística entrenada para la i-ésima clase.
    Debe establecer p en un vector de valores de 0..K-1 (por ejemplo, p = [0, 2, 0, 1]
    predice clases 0, 2, 0, 1 para 4 ejemplos).

    Parametros
    ----------
    all_theta : array_like
        The trained parameters for logistic regression for each class.
        This is a matrix of shape (K x n+1) where K is number of classes
        and n is number of features without the bias.

    X : array_like
        Data points to predict their labels. This is a matrix of shape
        (m x n) where m is number of data points to predict, and n is number
        of features without the bias term. Note we add the bias term for X in
        this function.

    Devuelve
    -------
    p : array_like
        The predictions for each data point in X. This is a vector of shape (m, ).
    """

    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p

In [87]:
print(X.shape)
pred = predictOneVsAll(all_theta, X)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y) * 100))
XPrueba = X[1:95487, :].copy()
print(XPrueba.shape)
#print(np.ones((1)))
#print(XPrueba)
#p = np.zeros(1)
XPrueba = np.concatenate([np.ones((95486, 1)), XPrueba], axis=1)
print(XPrueba.shape)
p = np.argmax(sigmoid(XPrueba.dot(all_theta.T)), axis = 1)
print(p)

# displayData(X[1002:1003, :])
print(y[1:95487])

(95487, 16)
Precision del conjuto de entrenamiento: 80.17%
(95486, 16)
(95486, 17)
[2 1 2 ... 0 1 2]
[2 1 2 ... 0 1 2]


In [88]:
#usamos x_train para comprobar que el aprendizaje predice los valores de y_train
pred = predictOneVsAll(all_theta, X)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y) * 100))
XPrueba = X_train[1:95487, :].copy()
print(XPrueba.shape)
#print(np.ones((1)))
#print(XPrueba)
#p = np.zeros(1)
# Normalizar XPrueba utilizando las medias y desviaciones estándar de X_train
XPrueba_norm = (XPrueba - mu) / sigma

# Agregar la columna de unos
XPrueba_norm_con_intercepto = np.concatenate([np.ones((XPrueba_norm.shape[0], 1)), XPrueba_norm], axis=1)

# Realizar predicciones en el conjunto de prueba normalizado
p = np.argmax(sigmoid(XPrueba_norm_con_intercepto.dot(all_theta.T)), axis=1)
print(p)

# displayData(X[1002:1003, :])
print(y_train[1:95487])

Precision del conjuto de entrenamiento: 80.17%
(40922, 16)
[1 1 2 ... 2 2 1]
['Elevated' 'Elevated' 'Hypertension Stage 1' ... 'Hypertension Stage 1'
 'Hypertension Stage 1' 'Hypertension Stage 1']
